[View in Colaboratory](https://colab.research.google.com/github/ebgv/Planet--Understanding-the-Amazon-from-Space/blob/master/main_Eric.ipynb)

# Planet: Understanding the Amazon from Space

## 0. Installing all the required packages

In [0]:
# to install pytorch on colab
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision

In [2]:
!pip install -U bcolz

    100% |████████████████████████████████| 1.5MB 6.6MB/s 
  Running setup.py bdist_wheel for bcolz ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / done
  Stored in directory: /root/.cache/pip/wheels/9f/78/26/fb8c0acb91a100dc8914bf236c4eaa4b207cb876893c40b745
Successfully built bcolz


In [3]:
!pip install Pillow==4.0.0

    100% |████████████████████████████████| 5.6MB 5.3MB/s 
torchvision 0.2.1 has requirement pillow>=4.1.1, but you'll have pillow 4.0.0 which is incompatible.
  Found existing installation: Pillow 5.3.0
    Uninstalling Pillow-5.3.0:
      Successfully uninstalled Pillow-5.3.0


In [0]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
import torch
import torch.nn as nn
import torchvision
from torchvision import models,transforms,datasets
import bcolz
import time
%matplotlib inline

## 1. Downloading the data

First, we create the directory where we will store the data:

In [5]:
%mkdir data
%cd data/

/content/data


Now we retrive the data from kaggle:

In [8]:
!pip install kaggle

from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth

auth.authenticate_user()
drive_service = build('drive', 'v3')
results = drive_service.files().list(q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])
filename = "/content/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)
request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

Download 100%.


In [21]:
!ls -a
import json
token = {"username":"","key":""}
with open("/content/.kaggle/kaggle.json", "w") as file:
    json.dump(token, file)
!chmod 600 /content/.kaggle/kaggle.json
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
!kaggle config set -n path -v{/content/data}
!kaggle competitions download -c planet-understanding-the-amazon-from-space

 .   ..  '{'
- path is now set to: {/content/data}
 98% 589M/600M [00:10<00:00, 75.5MB/s]
100% 600M/600M [00:10<00:00, 58.5MB/s]
100% 603M/603M [00:15<00:00, 6.34MB/s]
100% 603M/603M [00:15<00:00, 40.9MB/s]
100% 12.9G/12.9G [09:08<00:00, 12.7MB/s]

 74% 14.4G/19.4G [10:11<03:35, 25.3MB/s]
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 11, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.6/dist-packages/kaggle/cli.py", line 50, in main
    out = args.func(**command_args)
  File "/usr/local/lib/python3.6/dist-packages/kaggle/api/kaggle_api_extended.py", line 677, in competition_download_cli
    quiet)
  File "/usr/local/lib/python3.6/dist-packages/kaggle/api/kaggle_api_extended.py", line 644, in competition_download_files
    force, quiet)
  File "/usr/local/lib/python3.6/dist-packages/kaggle/api/kaggle_api_extended.py", line 622, in competition_download_file
    self.download_file(response, outfile, quiet)
  File "/usr/local/lib/python3.6/dist-pac

In [0]:
# Check files:
!cd /content/data

## 2.  Loading the data